In [34]:
##import necessary libraries
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe #parametr optimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [35]:
import mlflow
from mlflow.models import infer_signature

In [36]:
## load the dataset
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";",)
data.head

<bound method NDFrame.head of       fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.0              0.27         0.36            20.7      0.045   
1               6.3              0.30         0.34             1.6      0.049   
2               8.1              0.28         0.40             6.9      0.050   
3               7.2              0.23         0.32             8.5      0.058   
4               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4893            6.2              0.21         0.29             1.6      0.039   
4894            6.6              0.32         0.36             8.0      0.047   
4895            6.5              0.24         0.19             1.2      0.041   
4896            5.5              0.29         0.30             1.1      0.022   
4897            6.0              0.21         0.38             0.8      0.020  

In [ ]:
## Split the data into training,validation and test sets
train,test=train_test_split(data,test_size=0.25,random_state=42) #100-25=75% training and 25 for testing
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [38]:
#inspect the quality column
train[['quality']].values.ravel()


array([6, 6, 5, ..., 6, 6, 8], shape=(3673,))

In [39]:
#define the target and the independent variables
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

In [40]:
## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

In [41]:
## splitting this train data into train and validation
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=
0.20,random_state=42)
signature=infer_signature(train_x,train_y)

In [42]:
#checking the train_x
np.mean(train_x,axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [43]:
#inspecting to see the correct number of independent variables
train_x.shape[1]

11

In [47]:
# ANN Model definition
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Calculate mean and variance for normalization
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    # Define the model architecture
    model = keras.Sequential([
        keras.Input(shape=(train_x.shape[1],)),  # Input layer
        keras.layers.Normalization(mean=mean, variance=var),  # Normalize input data
        keras.layers.Dense(64, activation='relu'),  # Hidden layer with ReLU activation
        keras.layers.Dense(1)  # Output layer (for regression task)
    ])

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=params["lr"], momentum=params["momentum"]),
        loss="mean_squared_error",  # Loss function for regression
        metrics=[keras.metrics.RootMeanSquaredError()]  # RMSE as metric
    )
    
    # Train the model and log the run in MLflow
    with mlflow.start_run(nested=True):
        # Fit the model
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=epochs, batch_size=64)
        
        # Evaluate the model on the validation set
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]  # Extract RMSE from evaluation result
        
        # Log parameters, metrics, and model to MLflow
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)
        
        # Log the model to MLflow with signature
        mlflow.tensorflow.log_model(model, "model", signature=signature)
        
    # Return the result for hyperopt
    return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [48]:
# Objective function for Hyperopt optimization
def objective(params):
    result = train_model(
        params=params,
        epochs=3,  # Number of epochs
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result



In [49]:
# Define the search space for hyperparameter optimization
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),  # Learning rate, search on a log scale
    "momentum": hp.uniform("momentum", 0.0, 1.0)  # Momentum, search uniformly between 0 and 1
}

In [50]:
# Set up MLflow experiment
mlflow.set_experiment("wine-quality")

# Start the MLflow tracking run
with mlflow.start_run():
    # Conduct hyperparameter optimization using Hyperopt
    trials = Trials()
    
    # Run Hyperopt's fmin function to optimize the parameters
    best = fmin(
        fn=objective,  # The function to minimize (our objective function)
        space=space,  # The search space
        algo=tpe.suggest,  # Tree of Parzen Estimators optimization algorithm
        max_evals=4,  # Maximum number of evaluations (you can adjust this based on your preference)
        trials=trials  # Store the trial results in this variable
    )
    
    # Get the best run from the trials
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]
    
    # Log the best hyperparameters and the corresponding loss to MLflow
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    
    # Log the best model
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)
    
    # Print out the best hyperparameters and the corresponding RMSE loss
    print(f"Best parameters: {best}")
    print(f"Best eval RMSE: {best_run['loss']}")

2025/11/04 12:34:54 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2025-11-04 12:34:54.560759: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 25s 561ms/step - loss: 35.4329 - root_mean_squared_error: 5.9526
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28.4331 - root_mean_squared_error: 5.3192   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 21.3655 - root_mean_squared_error: 4.6223 - val_loss: 10.6943 - val_root_mean_squared_error: 3.2702

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 11s 245ms/step - loss: 11.4834 - root_mean_squared_error: 3.3887
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.4327 - root_mean_squared_error: 2.8966    
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.4877 - root_mean_squared_error: 2.5471 - val_loss: 3.9098 - val_root_mean_squared_error: 1.9773

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.0243 - root_mean_squared_error: 1.7391
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.3459 - root_mean_squared_error: 1.8

2025/11/04 12:34:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 338ms/step - loss: 38.5760 - root_mean_squared_error: 6.2110
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 36.4405 - root_mean_squared_error: 6.0356   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 33.1361 - root_mean_squared_error: 5.7564 - val_loss: 26.8787 - val_root_mean_squared_error: 5.1845

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 27.1291 - root_mean_squared_error: 5.2086
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24.8539 - root_mean_squared_error: 4.9837 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.4242 - root_mean_squared_error: 4.7354 - val_loss: 17.8396 - val_root_mean_squared_error: 4.2237

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 17.9566 - root_mean_squared_error: 4.2375
41/46 ━

2025/11/04 12:35:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:26<00:00,  6.69s/trial, best loss: 0.7415744662284851]

2025/11/04 12:35:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.01965893260581649), 'momentum': np.float64(0.7365258964767939)}
Best eval RMSE: 1.656072735786438
